This program scrapes data from boxofficemojo.com, puts it into a dataframe and cleans the dataframe to answer these questions:

1. What is the top-grossing movie in each season per year?

2. What is the top-grossing domestic movie for the whole year, per year?

3. What is the top-grossing movie of all time?

4. Which season do people watch the most movies?

In [139]:
## Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.display import display, HTML

In [140]:
## Main function
def main():

    ## Create dataframe
    #Information    Year    Movie    Cummulative Gross    Movies    Released    Average Gross
    #               Movie 1
    #               Movie 2
    # Winter        Movie 3
    #
    #               Movie 1
    #               Movie 2
    # Spring        Movie 3
    #etc...

    # Create dataframe
    seasonal_df = get_seasonal_data()
    display(HTML(seasonal_df.head(10).to_html().replace("\\n","<br>")))

    # Answer questions
    answer_questions(seasonal_df) 
    

In [141]:
## Method to get soup
def get_soup(url):

    # Get response
    response = requests.get(url)

    # Check response
    if response.status_code == 200:

        # Get soup
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
    
    else:

        # Print error
        print("Error: Response code", response.status_code)
        return None

In [142]:
## Method to get seasonal data
def get_seasonal_data():

    # Create list of seasons and seasonal dataframe
    seasons = ["winter", "spring", "summer", "fall", "holiday"]
    seasonal_df = pd.DataFrame(columns=["Season", "Year", "Movie", "Cummulative Gross", "Releases", "Average Gross"])

    # Loop through seasons to get soup from each season
    for season in seasons:

        # Create list of desired data for dataframe
        years = []
        money = []                  # Money on website has the same class for all three types of money, so this list holds everything  
        cummulativeGross = []
        releases = []
        averageGross = []
        movies = []

        # Base url
        url = "https://www.boxofficemojo.com/season/" + season + "/?grossesOption=calendarGrosses"

        # Get soup
        soup = get_soup(url)

        # Get data if soup exists
        if soup != None:

            # Year
            yearElements = soup.find_all("td", class_="a-text-left mojo-header-column mojo-truncate mojo-field-type-year mojo-sort-column")
            years = [element.getText() for element in yearElements]

            # Cumulative Gross
            moneyElements = soup.find_all("td", class_="a-text-right mojo-field-type-money")
            money = [element.getText() for element in moneyElements]
            cummulativeGross = money[0::3]   # Every third element is the cumulative gross, second and third are average and top release gross

            # Releases
            releasesElements = soup.find_all("td", class_="a-text-right mojo-field-type-positive_integer")
            releases = [element.getText() for element in releasesElements]

            # Average Gross
            averageGross = money[1::3]      # Every second of three element is the average gross, first and third are cumulative and top release gross

            # Movies
            moviesElements = soup.find_all("td", class_="a-text-left mojo-field-type-release mojo-cell-wide")
            movies = [element.getText() for element in moviesElements]

        # Add data to dataframe
        for year in years:
            seasonal_df.loc[len(seasonal_df)] = [season.capitalize(), year, movies[years.index(year)], cummulativeGross[years.index(year)], releases[years.index(year)], averageGross[years.index(year)]]

        # Combine seasons column
        seasonal_df = seasonal_df.groupby(seasonal_df["Season"].replace("", np.nan).ffill()).agg({"Year" : '\n'.join, "Movie" : '\n'.join, "Cummulative Gross" : '\n'.join, "Releases" : '\n'.join, "Average Gross" : '\n'.join}).reset_index().dropna(subset=["Season"])


    # Return dataframe
    return seasonal_df

In [143]:
## Method to answer questions
def answer_questions(seasonal_df):

    ## 1. What is the top-grossing movie in each season per year?
    print("The seasonal dataframe shows the top-grossing movie in each season per year.")


    ## 2. What is the top-grossing domestic movie for the whole year, per year? 

    # highestGrossingMovie = {year: [movie, income]} holds the highest grossing movie for each year
    highestGrossingMovie = {}

    # movieList = [{season: [movie, income, year]}] holds information for each movie in each season
    movieList = []

    # Loop through dataframe
    for row, season in seasonal_df.iterrows():
        years = season.Year.split("\n")
        for year in years:
            movie = season.Movie.split("\n")[years.index(year)]
            income = season["Cummulative Gross"].split("\n")[years.index(year)]
            movieList.append({season.Season: [movie, income, year]})

    # Loop through movieList to find highest grossing movie for each year
    for movie in movieList:
        season = list(movie.keys())[0]
        year = movie[season][2]
        income = movie[season][1]
        if year not in highestGrossingMovie:
            highestGrossingMovie[year] = [movie[season][0], income]
        elif int(income.replace("$", "").replace(",", "")) > int(highestGrossingMovie[year][1].replace("$", "").replace(",", "")):
            highestGrossingMovie[year] = [movie[season][0], income]

    # Print highest grossing movie for each year
    print("The highest grossing movie for each year is:")
    for year in highestGrossingMovie:
        print(year, ":", highestGrossingMovie[year][0])


    ## 3. What is the top-grossing movie of all time?
    allTimeGrossingMovie = ['test', 0]
    for year in highestGrossingMovie:
        if int(highestGrossingMovie[year][1].replace("$", "").replace(",", "")) > int(str(allTimeGrossingMovie[1]).replace("$", "").replace(",", "")):
            allTimeGrossingMovie = [highestGrossingMovie[year][0], highestGrossingMovie[year][1]]

    print("\n")
    print("The highest grossing movie of all time is", allTimeGrossingMovie[0], "with a gross of", allTimeGrossingMovie[1])
    print("\n")


    ## 4. Which season do people watch the most movies?
    # grossPerSeason = {season: total gross} holds the total gross for each season
    grossPerSeason = {}

    # Loop through dataframe
    for row, season in seasonal_df.iterrows():
        years = season.Year.split("\n")
        for year in years:
            seasonName = season.Season
            income = season["Cummulative Gross"].split("\n")[years.index(year)]
            if seasonName not in grossPerSeason:
                grossPerSeason[seasonName] = int(income.replace("$", "").replace(",", ""))
            else:
                grossPerSeason[seasonName] += int(income.replace("$", "").replace(",", ""))

    # Find season with highest gross
    highestGrossingSeason = ["test", 0]
    for season in grossPerSeason:
        if grossPerSeason[season] > highestGrossingSeason[1]:
            highestGrossingSeason = [season, grossPerSeason[season]]

    highestGrossingSeason[1] = "${:,}".format(highestGrossingSeason[1])
    print("The season with the highest gross is", highestGrossingSeason[0], "with a gross of", highestGrossingSeason[1])
        


In [144]:
## Run main
if __name__ == "__main__":
    main() 

,Season,Year,Movie,Cummulative Gross,Releases,Average Gross
0,Fall,2023202220212020201920182017201620152014201320122011201020092008200720062005200420032002200120001999199819971996199519941993199219911990198919881987198619851984198319821981198019791977,Taylor Swift: The Eras TourBlack AdamVenom: Let There Be CarnageTenetJokerVenomItSullyThe MartianGone GirlGravityHotel TransylvaniaThe Lion King2011 3D ReleaseJackass 3DCloudy with a Chance of MeatballsEagle EyeThe Game PlanThe DepartedFlightplanShark TaleScary Movie 3Sweet Home AlabamaTraining DayMeet the ParentsDouble JeopardyRush HourIn & OutThe First Wives ClubSe7enForrest GumpThe FugitiveThe Last of the MohicansFreddy's Dead: The Final NightmareGhostSea of LoveCocktailFatal AttractionCrocodile DundeeBack to the FutureAll of MeNever Say Never AgainAn Officer and a GentlemanContinental DividePrivate Benjamin10Star Wars: Episode IV - A New Hope,"$926,845,054$734,082,949$866,355,282$117,551,002$1,403,820,286$1,403,497,899$1,173,481,576$1,129,541,691$1,249,071,635$1,352,668,265$1,124,107,841$1,150,604,864$1,091,570,660$1,116,827,351$1,145,260,534$1,179,402,588$978,043,143$1,028,137,150$983,490,411$998,403,401$1,226,694,885$993,140,869$852,184,926$774,209,652$871,900,675$773,358,197$794,845,805$618,573,133$627,037,580$623,613,655$630,374,473$540,866,352$461,128,534$519,477,461$480,394,696$377,212,756$409,716,267$419,778,490$309,058,591$358,034,853$312,308,675$279,397,701$32,113,438$279,299,713$118,888,887$109,135,486",160164131100272318310297275308300281287253243271263262260255273258127146142827887807169686663546259514346383761743,"$5,792,781$4,476,115$6,613,399$1,175,510$5,161,103$4,413,515$3,785,424$3,803,170$4,542,078$4,391,780$3,747,026$4,094,679$3,803,382$4,414,337$4,713,006$4,352,039$3,718,795$3,924,187$3,782,655$3,915,307$4,493,387$3,849,383$6,710,117$5,302,805$6,140,145$9,431,197$10,190,330$7,110,036$7,837,969$8,783,290$9,135,861$7,953,916$6,986,795$8,245,673$8,896,198$6,084,076$6,944,343$8,230,950$7,187,409$7,783,366$8,218,649$7,551,289$5,352,239$16,429,394$29,722,221$36,378,495"
1,Holiday,20232022202120202019201820172016201520142013201220112010200920082007200620052004200320022001200019991998199719961995199419931992199119901989198819871986198519841983198219811980197919781977197619751972,The Hunger Games: The Ballad of Songbirds & SnakesAvatar: The Way of WaterSpider-Man: No Way HomeThe Croods: A New AgeFrozen IIThe GrinchStar Wars: Episode VIII - The Last JediRogue One: A Star Wars StoryStar Wars: Episode VII - The Force AwakensThe Hunger Games: Mockingjay - Part 1The Hunger Games: Catching FireSkyfallThe Twilight Saga: Breaking Dawn - Part 1Harry Potter and the Deathly Hallows: Part 1AvatarMadagascar: Escape 2 AfricaI Am LegendHappy FeetHarry Potter and the Goblet of FireThe IncrediblesThe Lord of the Rings: The Return of the KingHarry Potter and the Chamber of SecretsHarry Potter and the Sorcerer's StoneHow the Grinch Stole ChristmasToy Story 2The WaterboyTitanicRansomToy StoryThe Santa ClauseMrs. DoubtfireHome Alone 2: Lost in New YorkThe Addams FamilyHome AloneBack to the Future Part IITwinsThree Men and a BabyStar Trek IV: The Voyage HomeRocky IVBeverly Hills CopSudden ImpactTootsieRaiders of the Lost Ark9 to 5Star Trek: The Motion PictureSupermanClose Encounters of the Third KindKing KongJawsThe Poseidon Adventure,"$681,436,646$1,399,001,102$1,529,945,428$141,313,215$2,198,128,197$2,227,125,300$2,464,252,488$2,316,973,910$2,433,852,535$1,817,994,796$2,205,634,407$2,148,217,840$1,856,106,353$1,916,622,857$2,283,063,082$1,656,471,144$1,828,892,856$1,811,536,356$1,859,715,946$1,777,761,628$1,703,043,868$1,884,610,262$1,738,682,033$1,636,532,301$1,411,875,839$1,330,785,803$1,110,700,414$1,100,350,344$1,037,668,875$975,520,103$838,454,602$957,926,493$807,633,170$830,366,349$711,548,809$632,939,999$626,684,657$538,182,417$509,956,585$489,684,428$405,780,845$424,519,938$316,246,351$525,968,293$315,149,888$279,843,614$104,644,068$52,614,445$190,274,624$84,563,118",121142130

The seasonal dataframe shows the top-grossing movie in each season per year.
The highest grossing movie for each year is:
2023 : Barbie
2022 : Top Gun: Maverick
2021 : Black Widow
2020 : Bad Boys for Life
2019 : The Lion King
2018 : Incredibles 2
2017 : Wonder Woman
2016 : Finding Dory
2015 : Jurassic World
2014 : Guardians of the Galaxy
2013 : Iron Man 3
2012 : The Avengers
2011 : Harry Potter and the Deathly Hallows: Part 2
2010 : Toy Story 3
2009 : Transformers: Revenge of the Fallen
2008 : The Dark Knight
2007 : Spider-Man 3
2006 : Pirates of the Caribbean: Dead Man's Chest
2005 : Star Wars: Episode III - Revenge of the Sith
2004 : Shrek 2
2003 : Finding Nemo
2002 : Spider-Man
2001 : Shrek
2000 : Mission: Impossible II
1999 : Star Wars: Episode I - The Phantom Menace
1998 : Armageddon
1997 : Men in Black
1996 : Independence Day
1995 : Batman Forever
1994 : The Lion King
1993 : Jurassic Park
1992 : Batman Returns
1991 : Terminator 2: Judgment Day
1990 : Ghost
1989 : Batman
1988 : Wh